<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/mfml2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from functools import reduce

In [ ]:
def create_random_invertible_matrix(n=5):
    A = np.random.randint(low=0,high=10,size=(n, n))
    mx = np.sum(np.abs(A), axis=1)
    np.fill_diagonal(A, mx)
    A=A.astype(float)
    return A
def transpose(A):
    m=A.shape[0]
    n=A.shape[1]
    A_T=np.zeros((n,m))
    for i in range(n):
        A_T[i]=A[:,i]
    return A_T
def multiply(A,B):
    if A.shape[1] != B.shape[0]:
       print("Illegal multiplication")
       return
    m=A.shape[0]
    n=A.shape[1]
    p=B.shape[1]
    C=np.zeros((m,p))
    for i in range(m):
        for k in range(p):
            for j in range(n):
                C[i][k]+=A[i][j]*B[j][k]

    return C

def create_random_symmetric_positive_definite_matrix(n):
    A=create_random_invertible_matrix(n)
    A_T=transpose(A)
    return multiply(A,A_T)

def create_identity_matrix(n):
     I=np.zeros((n,n))
     for i in range(n):
         I[i][i]=1
     return I



#Elementary row operations
def const_multiple(A,i,c):
    A[i]*=c
def exchange_rows(A,i,j):
    A[[i,j]] = A[[j,i]]
def add_const_row_multiple(A,i,j,c):
    A[i]+=A[j]*c

def find_pivot_row(A, curr_pivot_row, col):
    rows = A.shape[0]
    for row in range(curr_pivot_row, rows):
        if A[row, col] != 0.0:
            return row
    return None




def convert_pivot_row_column_values_to_zero_below(A,pivot_row,col,elementary_matrices):
    n=A.shape[0]
    rows = A.shape[0]

    for row in range(pivot_row+1, rows):
        const_multiplier=A[row][col]/A[pivot_row][col]
        elementary_matrix=create_identity_matrix(n)
        elementary_matrix[row][col]=const_multiplier
        elementary_matrices.append(elementary_matrix)
        #print('subtracting {}*A[{}] from A[{}]'.format(const_multiplier, pivot_row,row))
        A[row] -=const_multiplier*A[pivot_row]
    return elementary_matrices

def calculate_row_echelon_form(A):
    n=A.shape[0]

    pivot_row_cols=[]
    pivot_cols=[]
    non_pivot_cols=[]
    rows=A.shape[0]
    cols=A.shape[1]
    elementary_matrices=[]
    permutation_matrices=[]


    curr_pivot_row=0
    for col in range(cols):
        nonzero_row = find_pivot_row(A, curr_pivot_row, col)
        if nonzero_row is not None:
            if curr_pivot_row != nonzero_row:
                exchange_rows(A,curr_pivot_row,nonzero_row)
                perm_matrix=create_identity_matrix(n)
                exchange_rows(perm_matrix,curr_pivot_row,nonzero_row)
                permutation_matrices.append(perm_matrix)
                perm_matrix_inverse=transpose(perm_matrix)
                elementary_matrices.append(perm_matrix_inverse)
            pivot_row_cols.append((curr_pivot_row,col))
            convert_pivot_row_column_values_to_zero_below(A,curr_pivot_row,col,elementary_matrices)
            curr_pivot_row+=1
        else:
            print('Stopping...')
            if len(elementary_matrices) == 0:
               elementary_matrices.append(create_identity_matrix(n))
            return elementary_matrices,permutation_matrices


    return elementary_matrices,permutation_matrices

def multiply_all_matrices(matrices):
    n=matrices[0].shape[0]
    I=create_identity_matrix(n)
    #product = reduce(lambda x, y: multiply(x,y), matrices,I)
    product=I
    for matrix in matrices:
        product=multiply(product,matrix)
    return product

In [ ]:

import copy
def calculate_LU_Decomposition(A_original):
    print('Input matrix is ')
    print(A_original)
    print('---------')
    A=copy.deepcopy(A_original)
    elementary_matrices,permutation_matrices=calculate_row_echelon_form(A)
    print('U is')
    print(A)
    print('-----')
    L=multiply_all_matrices(elementary_matrices)
    L_modified=L
    print('------')
    if len(permutation_matrices) >0:

       for permutation_matrix in permutation_matrices:
           L_modified=multiply(permutation_matrix,L_modified)

    L=L_modified
    print("L is")
    print(L)
    print('------')

    if len(permutation_matrices) >0:
        print('perm matrices are {}'.format(len(permutation_matrices)))
        P=multiply_all_matrices(permutation_matrices)
        print(P)
        print('**********')

    A_check=multiply(L,A)
    print('printing L*U')
    print(A_check)




In [ ]:
A=create_random_symmetric_positive_definite_matrix(n=3)
A=A.astype(float)
calculate_LU_Decomposition(A)

Input matrix is 
[[526. 169. 161.]
 [169. 209. 116.]
 [161. 116. 209.]]
---------
U is
[[526.         169.         161.        ]
 [  0.         154.70152091  64.27186312]
 [  0.           0.         133.01832303]]
-----
------
L is
[[1.         0.         0.        ]
 [0.32129278 1.         0.        ]
 [0.30608365 0.41545722 1.        ]]
------
3
3
3
3
printing L*U
[[526. 169. 161.]
 [169. 209. 116.]
 [161. 116. 209.]]


In [ ]:
A = np.array([[ 1 ,1,1 ], [1,1,3], [2,5,8]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[1. 1. 1.]
 [1. 1. 3.]
 [2. 5. 8.]]
---------
U is
[[1. 1. 1.]
 [0. 3. 6.]
 [0. 0. 2.]]
-----
------
L is
[[1. 0. 0.]
 [2. 1. 0.]
 [1. 0. 1.]]
------
perm matrices are 1
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
**********


In [ ]:
A = np.array([[ 10 ,45,0 ], [10,34,0], [10,118,0]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[ 10.  45.   0.]
 [ 10.  34.   0.]
 [ 10. 118.   0.]]
---------
Stopping...
U is
[[ 10.  45.   0.]
 [  0. -11.   0.]
 [  0.   0.   0.]]
-----
------
L is
[[ 1.          0.          0.        ]
 [ 1.          1.          0.        ]
 [ 1.         -6.63636364  1.        ]]
------


In [ ]:
A = np.array([[ 10 ,0,45 ], [10,0,34], [10,0,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)


Input matrix is 
[[ 10.   0.  45.]
 [ 10.   0.  34.]
 [ 10.   0. 118.]]
---------
Stopping...
U is
[[ 10.   0.  45.]
 [  0.   0. -11.]
 [  0.   0.  73.]]
-----
------
L is
[[1. 0. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
------


In [ ]:
A = np.array([[ 0,10,45 ], [0,10,34], [0,10,118]])
A=A.astype(float)
calculate_LU_Decomposition(A)

Input matrix is 
[[  0.  10.  45.]
 [  0.  10.  34.]
 [  0.  10. 118.]]
---------
Stopping...
U is
[[  0.  10.  45.]
 [  0.  10.  34.]
 [  0.  10. 118.]]
-----
------
L is
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
------


In [ ]:
A=create_random_symmetric_positive_definite_matrix(n=3)
A

array([[194., 211., 217.],
       [211., 277., 193.],
       [217., 193., 469.]])

In [ ]:
calculate_LU_Decomposition(A)

Input matrix is 
[[194. 211. 217.]
 [211. 277. 193.]
 [217. 193. 469.]]
---------
U is
[[ 1.94000000e+02  2.11000000e+02  2.17000000e+02]
 [ 2.84217094e-14  4.75103093e+01 -4.30154639e+01]
 [ 2.57327943e-14  0.00000000e+00  1.87327330e+02]]
-----
------
L is
[[ 1.          0.          0.        ]
 [ 1.08762887  1.          0.        ]
 [ 1.1185567  -0.90539221  1.        ]]
------


In [ ]:
def cholesky_decomposition(A):
    n=A.shape[0]
    L=np.zeros((n,n)).astype(float)
    for i in range(n):
        for j in range(i+1):
            if i==j:
               l_sum=0
               for k in range(j):
                    l_sum+=L[j][k]*L[j][k]
               L[i][j]=np.sqrt(A[i][j]-l_sum)
            else:
                l_sum=0
                for k in range(j):
                    l_sum+=L[i][k]*L[j][k]
                L[i][j]= (A[i][j]-l_sum)/L[j][j]
    return L,transpose(L)


In [ ]:
A=create_random_symmetric_positive_definite_matrix(n=3)
A=A.astype(float)
print('Input matrix is ')
print(A)
#calculate_LU_Decomposition(A)
L,L_T=cholesky_decomposition(A)
L,L_T

Input matrix is 
[[396. 312. 336.]
 [312. 528. 416.]
 [336. 416. 704.]]


(array([[19.89974874,  0.        ,  0.        ],
        [15.67858992, 16.79826831,  0.        ],
        [16.8846353 ,  9.00525724, 18.37972886]]),
 array([[19.89974874, 15.67858992, 16.8846353 ],
        [ 0.        , 16.79826831,  9.00525724],
        [ 0.        ,  0.        , 18.37972886]]))

In [ ]:
product=multiply_all_matrices([L,L_T])
print('L*L_T is')
print(product)
print('Original matrix was ')
print(A)

L*L_T is
[[396. 312. 336.]
 [312. 528. 416.]
 [336. 416. 704.]]
Original matrix was 
[[396. 312. 336.]
 [312. 528. 416.]
 [336. 416. 704.]]


In [ ]:
class Vector:
    def __init__(self,values):
        self.values=values
        self.dim=len(self.values)

    def norm(self):
        return np.sqrt(np.sum(self.values**2))

    def dot_product(self,that_vector):
        return np.sum(self.values*that_vector.values)

    def angle(self,that_vector):
        cos_theta=self.dot_product(that_vector)/(that_vector.norm()*self.norm())
        return 180*np.arccos(cos_theta)/np.pi

    def const_multiplier(self,c):

        copy_values=self.values*c
        my_copy=Vector(copy_values)
        return my_copy

    def project_to(self,that_vector):
        multiplier=self.dot_product(that_vector)/(that_vector.norm()**2)
        return that_vector.const_multiplier(multiplier)

    def subtract(self,that_vector):
        return Vector(np.subtract(self.values,that_vector.values))

    def get_unit_vector(self):
        return self.const_multiplier(1.0/self.norm())
    def __str__(self):
        print(','.join(str(self.values)))

a=Vector(np.array([3,4]))
b=Vector(np.array([-4,3]))
print(a.angle(b))
c=a.subtract(b)
#a.values,b.values,c.values
e=a.get_unit_vector()
e.values


90.0


array([0.6, 0.8])

In [ ]:
def calculate_QR_decomposition(A):
    m=A.shape[0]
    n=A.shape[1]
    Q=np.zeros((m,n))
    orthonormal_vectors=[]
    def take_out(q_v):
        if len(orthonormal_vectors) ==0:
            orthonormal_vectors.append(q_v.get_unit_vector())
            return

        for i in range(len(orthonormal_vectors)):
            that_vector=orthonormal_vectors[i]
            q_v=q_v.subtract(q_v.project_to(that_vector))
        orthonormal_vectors.append(q_v.get_unit_vector())


    for i in range(n):
        q=A[:,i]
        q_v=Vector(q)
        take_out(q_v)
        #print(orthonormal_vectors[i].values)
        Q[:,i]=orthonormal_vectors[i].values
    Q_T=transpose(Q)

    R=multiply(transpose(Q),A)
    return Q,R
A = np.array([[ 1 ,-6 ], [4,-9],[7,6]])
print(A)
calculate_QR_decomposition(A)



[[ 1 -6]
 [ 4 -9]
 [ 7  6]]


(array([[ 0.12309149, -0.48507125],
        [ 0.49236596, -0.72760688],
        [ 0.86164044,  0.48507125]]),
 array([[ 8.12403840e+00, -8.88178420e-16],
        [ 8.88178420e-16,  1.23693169e+01]]))

In [ ]:
A=create_random_symmetric_positive_definite_matrix(n=3)


In [ ]:
print(A)

[[ 54.  43.  74.]
 [ 43. 262.  75.]
 [ 74.  75. 202.]]


In [ ]:
calculate_QR_decomposition(A)

(array([[ 0.5336083 , -0.28829444, -0.79507767],
        [ 0.42491031,  0.90422791, -0.04269798],
        [ 0.731241  , -0.31505271,  0.6050028 ]]),
 array([[ 1.01197826e+02,  1.89114734e+02,  2.19065971e+02],
        [ 7.10542736e-15,  2.00882098e+02, -1.71573421e+01],
        [-2.84217094e-14, -3.55271368e-14,  6.01724695e+01]]))

In [ ]:
B=create_random_symmetric_positive_definite_matrix(n=5)
print(B)
U,_,_=np.linalg.svd(B)
a=Vector(U[:,0])
b=Vector(U[:,4])
C=U[:,[0,1,2,3]]
print("random 5 x 4 matrix having all its columns as linearly independent is as follows")
print(C)
print(C.shape)
Q,R=calculate_QR_decomposition(C)
print("Q is")
print(Q)
print("R is")
print(R)
print("Q*R gives back original matrix C")
print(multiply(Q,R))
_,Sigma,_=np.linalg.svd(C)
print("Sigma of U is ")
print(Sigma)


[[ 768.  402.  286.  402.  102.]
 [ 402. 1091.  345.  576.  218.]
 [ 286.  345.  514.  191.  152.]
 [ 402.  576.  191.  847.  262.]
 [ 102.  218.  152.  262.  232.]]
random 5 x 4 matrix having all its columns as linearly independent is as follows
[[-0.4365991   0.77907288  0.17541481 -0.36651601]
 [-0.63316556 -0.46909644 -0.4432114  -0.42299627]
 [-0.30260792  0.3287435  -0.55388417  0.62179756]
 [-0.52686445 -0.22743924  0.67904573  0.27704879]
 [-0.19830504 -0.11486111  0.07001802  0.47260123]]
(5, 4)
Q is
[[-0.4365991   0.77907288  0.17541481 -0.36651601]
 [-0.63316556 -0.46909644 -0.4432114  -0.42299627]
 [-0.30260792  0.3287435  -0.55388417  0.62179756]
 [-0.52686445 -0.22743924  0.67904573  0.27704879]
 [-0.19830504 -0.11486111  0.07001802  0.47260123]]
R is
[[ 1.00000000e+00 -2.35922393e-16 -9.02056208e-17  2.77555756e-17]
 [ 1.04083409e-17  1.00000000e+00 -1.02348685e-16  0.00000000e+00]
 [-1.49186219e-16 -1.21430643e-17  1.00000000e+00  1.59594560e-16]
 [ 9.71445147e-17 -2.08